In [1]:
!pip install pandas
!pip install -U scikit-learn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

root = 'data'      # csv数据放到data路径下
save = 'data_txt'  # txt保存到data_txt路径下
fetch_line = 128   # 每32行生成一个txt, 如果最后一个文件不够32行就不要了
max_length = 128   # 补全至512, 注意max_length必须大于字段总长

#* 创建文件目录
if not os.path.exists(save):
    os.makedirs(save)

# data目录下所有文件
files = os.listdir(root) 

# 不需要的字段， 可以自行添加
ignore_titles = [
    '@timestamp', '_@timestamp', '@version', '_index', '_score', '_type', 'bytes',
    'url', 'tags', 'ts.end', 'ts.start', 'type'

]


# 1. 找所有字段
col_titles = []
for ind, file in enumerate(files):
    # 文件路径
    file_path = os.path.join(root, file)  
    # 读取csv文件的内容
    contents = pd.read_csv(file_path, header=None, low_memory=False)
    # 列名
    titles   = contents.loc[0].values.tolist()  
    col_titles += titles

# 所有字段(无重复)
col_titles = sorted(list(set(col_titles)))
# 移除不需要的字段
col_titles = [c for c in col_titles if c not in ignore_titles]
# 保存有效字段
with open("titles.txt", 'w') as f:
    save_format = '\n'.join(col_titles)
    f.write(save_format)


# 2. 生成txt
# 使用OneHotEncoder进行独热编码
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")

# 遍历所有文件
for ind, file in enumerate(files):
    file_path = os.path.join(root, file)
    if not os.path.isfile(file_path):
        continue
    file_name = os.path.basename(file)[:-4]  # 文件名
    save_dir = os.path.join(save, file_name) # 以文件名命名的保存目录
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 读取csv文件的内容
    contents = pd.read_csv(file_path, header=None, low_memory=False)
    titles   = contents.loc[0].values.tolist()  # 列名
    # 移除不需要的字段
    titles_valid = [i for i, t in enumerate(titles) if t not in ignore_titles]

    # 所有特征
    # 空值转成字符
    contents.fillna('', inplace=True)
    data = contents[titles_valid]
    
    # 太大了会爆内存，这里每200个特征做一次onehot
    n_gap  = 16
    n_loop = math.ceil(len(data) / n_gap)
    counts = 0
    features = None
    for i in range(0, n_loop):
        if i == n_loop:
            a = data[ i*n_gap : -1]
        else:
            a = data[ i*n_gap : (i+1) * n_gap]
        encoded_features = encoder.fit_transform(a)
        if encoded_features.shape[0] < n_gap:
            continue
        encoded_features = encoded_features.astype('int')
        
        # 补齐0
        diff_len = max_length - encoded_features.shape[1]
        if diff_len <= 0:
            continue
        zero_matrix = np.zeros((encoded_features.shape[0], diff_len), dtype='int')
        padding_features = np.concatenate([encoded_features, zero_matrix], axis=1)

        if features is None:
            features = padding_features
        else:
            features = np.concatenate([features, padding_features])

        if features.shape[0] == fetch_line:
            # 保存txt文件
            save_to = os.path.join(save_dir, '{}.txt'.format(str(counts)))
            np.savetxt(save_to, features, fmt='%d')
            counts += 1
            features = None

/environment/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/environment/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/environment/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/environment/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in 

In [ ]:
import torch
from torchvision import datasets, transforms
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

from resnet import resnet50 as self_resnet50

import os
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 自定义数据集类
class MyDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)
        self.file_paths = []
        self.labels = []
        for i, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            files = os.listdir(class_path)
            self.file_paths.extend([os.path.join(class_path, file) for file in files])
            self.labels.extend([i] * len(files))

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        data = np.loadtxt(file_path).reshape(32, 512)
        data = torch.from_numpy(data).float()
        return data, label
    
    def get_info(self, idx):
        """ 返回文件路径和标签 """
        return self.file_paths[idx], self.labels[idx]

# 主函数
if __name__ == "__main__":
    # 设置随机种子
    torch.manual_seed(42)

    # 数据集目录和文件路径
    data_dir = 'data_txt/'

    # 构建数据集
    dataset = MyDataset(data_dir)

    # 划分训练集和测试集
    train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)

    # 创建数据加载器
    batch_size = 128
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # 实例化ResNet50模型
    resnet50 = self_resnet50(num_classes=5)

    # 指定设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    resnet50 = resnet50.to(device)

    # 定义损失函数和优化器
    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(resnet50.parameters(), lr=0.01, momentum=0.9, nesterov=True)

    # 训练参数
    epoch = 1

    # 训练循环
    for i in range(epoch):
        print("-------第{}轮训练开始-------".format(i + 1))
        resnet50.train()
        for data in tqdm(train_dataloader, ncols=100, desc='Train'):
            imgs, labels = data
            imgs = imgs.to(device).reshape(-1, 1, 32, 512)
            labels = labels.to(device)
            outputs = resnet50(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("训练次数：{}, Loss: {:.4f}".format(i + 1, loss.item()))

    # 提取训练数据的特征
    train_features = []
    train_labels = []
    resnet50.eval()
    for data in tqdm(train_dataloader, ncols=100, desc='Extracting Train Features'):
        imgs, labels = data
        imgs = imgs.to(device).reshape(-1, 1, 32, 512)
        with torch.no_grad():
            features = resnet50.extract_features(imgs)  # 假设此方法存在
        train_features.extend(features.cpu().numpy())
        train_labels.extend(labels.numpy())

    # 训练SVM分类器
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(train_features, train_labels)

    # 提取测试数据的特征并进行分类
    test_features = []
    test_labels = []
    for data in tqdm(test_dataloader, ncols=100, desc='Extracting Test Features'):
        imgs, labels = data
        imgs = imgs.to(device).reshape(-1, 1, 32, 512)
        with torch.no_grad():
            features = resnet50.extract_features(imgs)
        test_features.extend(features.cpu().numpy())
        test_labels.extend(labels.numpy())

    # 使用SVM分类器进行预测
    svm_predictions = svm_classifier.predict(test_features)

    # 计算准确率
    accuracy = accuracy_score(test_labels, svm_predictions)
    print(f"SVM Accuracy: {accuracy:.4f}")

    # 结果写入文件
    with open('result.txt', 'w') as f:
        f.write(f"SVM Accuracy: {accuracy:.4f}\n")
